In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate

from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import QuantileTransformer
import lightgbm as lgbm
import xgboost as xg
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import svm


In [ ]:
# this csv file was created from code that exists in the final.py
# originally, i had the preprocessing code and csv in its own separate directory
data = pd.read_csv('../code/preprocessing/punts_one_hot_encoded.csv')

In [ ]:
data = data.drop(['gameId', 'playId', 'missedTackler', 'assistTackler', 'tackler', 'gunners', 'puntRushers',
'specialTeamsResult', 'season', 'gameDate', 'gameTimeEastern', 'homeTeamAbbr', 'visitorTeamAbbr',
'playDescription', 'possessionTeam', 'specialTeamsPlayType', 'specialTeamsResult', 'kickerId', 'returnerId', 'kickBlockerId',
'yardlineSide', 'penaltyJerseyNumbers', 'penaltyCodes', 'playResult',
'kickDirectionIntended_L', 'kickDirectionIntended_R', 'kickDirectionIntended_C',
'returnDirectionIntended_C', 'returnDirectionIntended_L', 'returnDirectionIntended_R',
'specialTeamsSafeties', 'vises', 'penaltyYards', 'passResult', 'return_team_min', 'gameClock', 'down'], axis=1)

In [ ]:
data['kickReturnYardage'].isnull().sum().sum()

In [ ]:
data = data.dropna()

Box plot of kick return values

In [ ]:
kry = data['kickReturnYardage']
new_kry = kry[kry.values < 26]
plt.boxplot(new_kry.values)

In [ ]:
# get rid of values in dataframe where kick return yardage is > 25
data = data[data['kickReturnYardage'] < 25]

In [ ]:
# we'll use the same x values as what got me the highest score in the linear regression model
x = data.loc[:, ['hangTime', 'snapTime', 'operationTime', 'kickLength', 'yardsToGo', 'yardlineNumber',  'kicking_team__mean', 'kicking_team_std', 'kicking_team_min',
       'return_team__mean', 'return_team_std', 'kickType_A',
       'kickType_N', 'kickDirectionActual_C', 'kickDirectionActual_L',
       'kickDirectionActual_R', 'returnDirectionActual_C',
       'returnDirectionActual_L', 'returnDirectionActual_R', 'snapDetail_<', 'snapDetail_>',
       'snapDetail_H', 'snapDetail_L', 'snapDetail_OK']]

# data normalization
x = (x - np.mean(x, axis=0)) / np.std(x, axis=0)
x = x.values



In [ ]:
# target variable
y = data['kickReturnYardage'].values
# need to encode y into categories like i did before the SOM
# initialize label encoder class
label_encoder = LabelEncoder()
# set the bins
bins = pd.cut(y, [-20, 0, 10, 20, 150], retbins=True)
# have to change the bins now - will only be 3: less than 20, 1-10, 10-20
# bins = pd.cut(y, [-20, 0, 10, 20], retbins=True)
# encode the y values
encoded_y = label_encoder.fit_transform(bins[0])

In [ ]:
#split the dataset into training (70%) and testing (30%) sets
X_train,X_test,y_train,y_test = train_test_split(x,encoded_y,test_size=0.3,random_state=0)  

In [ ]:
#instantiate the model
log_regression = LogisticRegression()

#fit the model using the training data
log_regression.fit(X_train,y_train)

#use model to make predictions on test data
y_pred = log_regression.predict(X_test)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Applying cross validation

In [ ]:
cv_results = cross_validate(log_regression, x, encoded_y, cv=3)
cv_results['test_score']


In [ ]:
cv_results = cross_validate(log_regression, x, encoded_y, cv=10)
cv_results['test_score']
